In [1]:
%pip install --upgrade pip 
%pip install sqlalchemy
%pip install sqlalchemy_utils
%pip install psycopg2
%pip install faker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
from enum import Enum
from typing import List
from uuid import UUID, uuid4
from datetime import datetime
from contextlib import closing
from time import time
from random import choice as random_choice

from sqlalchemy import (
    create_engine,
    inspect,
    ForeignKey,
    UniqueConstraint,
    types
)
from sqlalchemy.orm import Mapped, mapped_column, relationship, validates, declarative_base, sessionmaker
from sqlalchemy_utils import database_exists, create_database

from faker import Faker

In [22]:
Base = declarative_base()



class LikeDislike(Enum):
    like = 1
    dislike = -1


class Review(Base):
    __tablename__ = "review"
    __table_args__ = (
        UniqueConstraint('user_id', 'movie_id', name='unique_review_user_movie'),
    )

    id: Mapped[UUID] = mapped_column(primary_key=True, default=uuid4)

    user_id: Mapped[UUID]
    movie_id: Mapped[UUID] = mapped_column(index=True)

    score: Mapped[int] = mapped_column(nullable=False, comment="Оценка отзыва")
    text: Mapped[str] = mapped_column(nullable=False, comment="Текст отзыва")
    dt: Mapped[datetime] = mapped_column(nullable=False)
    is_delete: Mapped[bool] = mapped_column(nullable=False)

    reviews_ratings: Mapped[List['UserReviewRating']] = relationship(
        back_populates="review", cascade='all, delete')
    
    @validates('score')
    def validate_score(self, key, value):
        if 0 <= value <= 10:
            return value
        raise ValueError("Value should be between 0 and 10")



class Bookmark(Base):
    __tablename__ = "bookmark"
    __table_args__ = (
        UniqueConstraint('user_id', 'movie_id', name='unique_bookmark_user_movie'),
    )

    id: Mapped[UUID] = mapped_column(primary_key=True, default=uuid4)

    user_id: Mapped[UUID] = mapped_column(index=True)
    movie_id: Mapped[UUID] = mapped_column(types.Uuid)
    dt: Mapped[datetime] = mapped_column(nullable=False)
    is_delete: Mapped[bool] = mapped_column(nullable=False)


class UserReviewRating(Base):
    __tablename__ = "user_review_rating"
    __table_args__ = (
        UniqueConstraint('user_id', 'review_id', name='unique_user_review_rating'),
    )
    id: Mapped[UUID] = mapped_column(primary_key=True, default=uuid4)

    user_id: Mapped[UUID] = mapped_column(index=True)

    review: Mapped["Review"] = relationship(back_populates="reviews_ratings")
    review_id: Mapped[UUID] = mapped_column(ForeignKey("review.id"))

    score: Mapped[LikeDislike]
    dt: Mapped[datetime] = mapped_column(nullable=False)
    is_delete: Mapped[bool] = mapped_column(nullable=False)



In [24]:

user = 'bench_user'
password = 'bench_pass'
host = 'localhost'
port = 5434
db = 'bench_db'

pg_url = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'
if not database_exists(pg_url):
    print('No db')
    create_database(pg_url)
    print('db created')

engine = create_engine(pg_url)

inspector = inspect(engine)
db_tables = inspector.get_table_names()

if not db_tables:
    print('Моделей нет в БД')
    Base.metadata.create_all(bind=engine)
    print('Модели добавлены в БД')
else:
    print('Моделей уже есть в БД')

Моделей уже есть в БД


In [18]:
fake: Faker = Faker()

USERS_COUNT = 500
USERS_IDS = [uuid4() for _ in range(USERS_COUNT)]

MOVIES_COUNT = 100
MOVIES_IDS = [uuid4() for _ in range(MOVIES_COUNT)]

session_factory = sessionmaker(bind=engine, expire_on_commit=False)

In [19]:
# insert review
with closing(session_factory()) as session:
    start_time: float = time()
    for user_id in USERS_IDS:
        for movie_id in MOVIES_IDS:
            new_review = Review(
                user_id=user_id,
                movie_id=movie_id,
                score=fake.random_int(min=0, max=10),
                text=fake.text(), # fake.paragraph,
                is_delete=False,
                dt=fake.date_time_between(start_date="-1y", end_date="now")
            )

            session.add(new_review)
            session.commit()

    insertion_time: float = time() - start_time
    insertion_speed: int = (USERS_COUNT * MOVIES_COUNT) // insertion_time

    print(f' -- Insert {USERS_COUNT * MOVIES_COUNT} reviews. Speed {insertion_speed} records/sec')

 -- Insert 50000 reviews. Speed 310.0 records/sec


In [20]:
# insert bookmarks
with closing(session_factory()) as session:
    start_time: float = time()
    for user_id in USERS_IDS:
        for movie_id in MOVIES_IDS:
            new_bookmark = Bookmark(
                user_id=user_id,
                movie_id=movie_id,
                is_delete=False,
                dt=fake.date_time_between(start_date="-1y", end_date="now")
            )

            session.add(new_bookmark)
            session.commit()

    insertion_time: float = time() - start_time
    insertion_speed: int = (USERS_COUNT * MOVIES_COUNT) // insertion_time

    print(f' -- Insert {USERS_COUNT * MOVIES_COUNT} booksmarks. Speed {insertion_speed} records/sec')

 -- Insertd: 50000 booksmarks. Speed 334.0 records/sec


In [25]:
# insert review ratings
with closing(session_factory()) as session:

    reviews_count_for_bench = 100
    users_count_for_bench = 500

    query_result = session.query(Review.id).limit(reviews_count_for_bench).all()
    all_reviews_ids = [review.id for review in query_result]
    start_time: float = time()
    for user_id in USERS_IDS[:users_count_for_bench]:
        for review_id in all_reviews_ids:
            score = random_choice([LikeDislike.like, LikeDislike.dislike])
            new_review_rating = UserReviewRating(
                user_id=user_id,
                review_id=review_id,
                score=score,
                is_delete=False,
                dt=fake.date_time_between(start_date="-1y", end_date="now")
            )

            session.add(new_review_rating)
            session.commit()

    insertion_time: float = time() - start_time
    insertion_speed: int = (reviews_count_for_bench * users_count_for_bench) // insertion_time

    print(f' -- Insert {reviews_count_for_bench * users_count_for_bench} reviews ratings. Speed {insertion_speed} records/sec')

Insert 50000 reviews ratings. Speed 308.0 records/sec


In [28]:
# select reviews
with closing(session_factory()) as session:
    start_time: float = time()
    all_reviews = session.query(Review).all()

    selection_time: float = time() - start_time

    all_reviews_count = len(all_reviews)
    selection_speed: int = all_reviews_count // selection_time

    print(f' -- Select {all_reviews_count} reviews. Speed: {selection_speed} records/sec')

 -- Selection 50000 reviews. Speed: 42303.0 records/sec


In [29]:
# select bookmarks
with closing(session_factory()) as session:
    start_time: float = time()
    all_booksmarks = session.query(Bookmark).all()

    selection_time: float = time() - start_time

    all_booksmarks_count = len(all_booksmarks)
    selection_speed: int = all_booksmarks_count // selection_time

print(f' -- Select {all_reviews_count} bookmarks. Speed: {selection_speed} records/sec')

 -- Selection 50000 bookmarks. Speed: 45083.0 records/sec


In [31]:
# select review ratings
with closing(session_factory()) as session:
    start_time: float = time()
    all_reviews_ratings = session.query(UserReviewRating).all()

    selection_time: float = time() - start_time

    all_reviews_ratings_count = len(all_reviews_ratings)
    selection_speed: int = all_reviews_ratings_count // selection_time

    print(f' -- Select {all_reviews_ratings_count} reviews ratings. Speed: {selection_speed} records/sec')

 -- Select 50000 reviews ratings. Speed: 45752.0 records/sec


In [32]:
# update review
with closing(session_factory()) as session:
    start_time: float = time()

    for user_id in USERS_IDS:
        for movie_id in MOVIES_IDS:
            review_to_update = session.query(Review).filter(Review.user_id==user_id,
                                                            Review.movie_id==movie_id).one_or_none()
            
            review_to_update.score=fake.random_int(min=0, max=10)
            review_to_update.text=fake.paragraph(), # fake.paragraph,
            review_dt=fake.date_time_between(start_date="-1y", end_date="now")

            session.commit()

    updation_time: float = time() - start_time
    updation_speed: int = (USERS_COUNT * MOVIES_COUNT) // updation_time

    print(f' -- Update {USERS_COUNT * MOVIES_COUNT} reviews. Speed: {updation_speed} records/sec')

 -- Update 50000 reviews. Speed: 228.0 records/sec
